<a href="https://colab.research.google.com/github/hiteshkukreja100/Call_Quality_Analyser/blob/main/Call_Quality_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Audio from Youtube

In [1]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 934.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.4 MB/s eta 0:00:00


In [10]:
import yt_dlp
import os

def download_audio_from_youtube(url, output_dir="downloads"):
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "%(id)s.%(ext)s")

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'quiet': True,
        'postprocessors': [
            {'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav', 'preferredquality': '192'},
        ],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        audio_path = os.path.join(output_dir, f"{info['id']}.wav")

    return audio_path

In [11]:
youtube_url = "https://www.youtube.com/watch?v=4ostqJD3Psc"  # enter your youtube url

raw_audio = download_audio_from_youtube(youtube_url)
print(f"Raw audio saved at: {raw_audio}")

Raw audio saved at: downloads/4ostqJD3Psc.wav


## Audio Pre Processing

In [4]:
!pip install librosa soundfile

In [12]:
import time
import torch
import torchaudio
import librosa
import numpy as np
import soundfile as sf

# Method 1: The torchaudio-based preprocessing function
def preprocess_audio_torchaudio(input_wav, target_sr=16000):
    # Load audio
    audio, sr = torchaudio.load(input_wav)

    # Convert to mono
    if audio.shape[0] > 1:
        audio = torch.mean(audio, dim=0, keepdim=True)

    # Resample
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(sr, target_sr)
        audio = resampler(audio)

    # Normalize
    audio = audio / audio.abs().max()

    # Save preprocessed file
    preprocessed_path = input_wav.replace(".wav", f"_preprocessed_{target_sr}.wav")
    torchaudio.save(preprocessed_path, audio, target_sr)

    return preprocessed_path

In [13]:
# Method 2: The more efficient librosa-based preprocessing function
def preprocess_audio_librosa(input_wav, target_sr=16000):
    # Load, resample, and convert to mono in one step
    audio, sr = librosa.load(input_wav, sr=target_sr, mono=True)

    # Trim silence
    audio, _ = librosa.effects.trim(audio)

    # Convert to a PyTorch tensor
    audio_tensor = torch.from_numpy(audio)

    # Save preprocessed file
    preprocessed_path = input_wav.replace(".wav", f"_preprocessed_{target_sr}_librosa.wav")
    sf.write(preprocessed_path, audio, target_sr)

    return preprocessed_path


In [14]:
# enter your downloaded audio path
audio_file_path = raw_audio

# --- Test the torchaudio method ---
start_time_torch = time.time()
preprocess_audio_torchaudio(audio_file_path)
end_time_torch = time.time()
torch_duration = end_time_torch - start_time_torch
print(f"torchaudio method took: {torch_duration:.4f} seconds")

# --- Test the librosa method ---
start_time_librosa = time.time()
preprocess_audio_librosa(audio_file_path)
end_time_librosa = time.time()
librosa_duration = end_time_librosa - start_time_librosa
print(f"librosa method took: {librosa_duration:.4f} seconds")

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

torchaudio method took: 0.4563 seconds
librosa method took: 0.2959 seconds


In [15]:
if torch_duration < librosa_duration:
    print("The torchaudio method was faster.")
else:
    print("The librosa method was faster.")

The librosa method was faster.
